In [2]:
import json
import os

# import the data from File
source_file = os.path.abspath(os.path.join(os.getcwd(), '..', 'example', 'json', 'drawflow_test.txt'))
debug_file = os.path.abspath(os.path.join(os.getcwd(), '..', 'example', 'json', 'drawflow_debug.json'))

with open(source_file, 'r') as f:
    json_data = json.load(f)

json_data = json.loads(json_data)

with open(debug_file, 'w') as f:
    f.write(json.dumps(json_data, indent=4))


In [5]:
from ensys.components import EnFlow, EnInvestment, EnConverter, EnSink, EnSource, EnGenericStorage, EnBus, \
    EnEnergysystem


def create_io_data(json_component) -> (dict, dict):
    # build component_data["inputs"]
    component_data = json_component["data"]

    input_data = {}
    print(f"number of inputs: {len(json_component['inputs'])}")
    print(f"number of outputs: {len(json_component['outputs'])}")

    if len(json_component["inputs"]) > 0:
        for input_name in json_component["inputs"]:
            print(f"input_name: {input_name}")
            target_bus_id = json_component["inputs"][input_name]['connections'][0]["node"]
            target_bus_name = json_data[target_bus_id]["name"]

            # flow_data = component_data["connections"]["inputs"][input_name]["formInfo"]
            for input in component_data["connections"]["inputs"]:
                if input_name in input:
                    flow_data = input[input_name]["formInfo"]

                    if flow_data["investment"] is True:
                        flow_data["nominal_value"] = EnInvestment(
                            maximum=flow_data["maximum"],
                            minimum=flow_data["minimum"],
                            ep_costs=flow_data["ep_costs"],
                            existing=flow_data["existing"],
                            nonconvex=flow_data["nonconvex"],
                            offset=flow_data["offset"],
                            overall_maximum=flow_data["overall_maximum"],
                            overalL_minimum=flow_data["overall_minimum"],
                            lifetime=flow_data["lifetime"],
                            age=flow_data["age"],
                            interest_rate=flow_data["interest_rate"],
                            fixed_costs=flow_data["fixed_costs"],
                            # custom_attributes="to be done"
                        )

            input_data[target_bus_name] = EnFlow(**flow_data)

    # build component_data["outputs"]
    output_data = {}

    if len(json_component["outputs"]) > 0:
        for output_name in json_component["outputs"]:
            target_bus_id = json_component["outputs"][output_name]['connections'][0]["node"]
            target_bus_name = json_data[target_bus_id]["name"]

            # print(f"target_bus_name: {target_bus_name}")
            # print(f"output_name: {output_name}")

            for output in component_data["connections"]["outputs"]:
                # print(f"output: {output}")
                if output_name in output:
                    flow_data = output[output_name]["formInfo"]
                    print(f"flow_data: {flow_data}")

                    if flow_data["investment"] is True:
                        flow_data["nominal_value"] = EnInvestment(
                            maximum=flow_data["maximum"],
                            minimum=flow_data["minimum"],
                            ep_costs=flow_data["ep_costs"],
                            existing=flow_data["existing"],
                            nonconvex=flow_data["nonconvex"],
                            offset=flow_data["offset"],
                            overall_maximum=flow_data["overall_maximum"],
                            overalL_minimum=flow_data["overall_minimum"],
                            lifetime=flow_data["lifetime"],
                            age=flow_data["age"],
                            interest_rate=flow_data["interest_rate"],
                            fixed_costs=flow_data["fixed_costs"],
                            # custom_attributes="to be done"
                        )

            output_data[target_bus_name] = EnFlow(**flow_data)

    return input_data, output_data


def build_conversion_factors(json_component) -> dict:
    component_ports = json_component["data"]["ports"]
    conversion_factors = {}

    # build conversion_factors
    for input_port in component_ports["inputs"]:
        input_name = component_ports["inputs"][input_port["id"]]["code"]

        target_bus_id = json_component["inputs"][input_name]['connections'][0]["node"]
        target_bus_name = json_data[target_bus_id]["name"]

        conversion_value = component_ports["inputs"][input_port["id"]]["number"]
        if conversion_value is not None:
            conversion_factors[target_bus_name] = conversion_value

    for output_port in component_ports["outputs"]:
        output_name = component_ports["outputs"][output_port["id"]]["code"]

        target_bus_id = json_component["outputs"][output_name]['connections'][0]["node"]
        target_bus_name = json_data[target_bus_id]["name"]

        conversion_value = component_ports["outputs"][output_port["id"]]["number"]
        if conversion_value is not None:
            conversion_factors[target_bus_name] = conversion_value

    return conversion_factors


def convert_gui_json_to_ensys(json_data: dict) -> (dict, dict):
    ensys_es = EnEnergysystem()

    for component_index in json_data:
        # print(f"Component: {json_data[json_index]['name']}")
        json_component = json_data[component_index]

        component_class = json_component["class"]
        component_name = json_component["name"]
        component_data = json_component["data"]

        if component_class != "bus":
            print(f"component_class: {component_name}")
            input_data, output_data = create_io_data(json_component)
            component_data["inputs"] = input_data
            component_data["outputs"] = output_data

        if component_class in ["converter", "transformer"]:
            conversion_factors = build_conversion_factors(json_component)
            component_data["conversion_factors"] = conversion_factors

            ensys_component = EnConverter(**component_data)

        elif component_class == "sink":
            ensys_component = EnSink(**component_data)

        elif component_class == "source":
            ensys_component = EnSource(**component_data)

        elif component_class == "genericStorage":
            print(f"component_data: {component_data}")
            ensys_component = EnGenericStorage(**component_data)

        elif component_class == "bus":
            ensys_component = EnBus(label=component_name)

        else:
            print(f"Component {component_name} is not supported yet")

        ensys_es.add(ensys_component)

    return ensys_es


es_new = convert_gui_json_to_ensys(json_data)

filepath = os.path.abspath(os.path.join(os.getcwd(), '..', 'example', 'json', 'converted_es.json'))
print(filepath)
with open(filepath, 'w+') as f:
    json.dump(es_new.model_dump(), f, indent=4)


component_class: wind
number of inputs: 0
number of outputs: 1
flow_data: {'investment': True, 'nominal_value': EnInvestment(maximum=None, minimum=0.0, ep_costs=116102.473, existing=0.0, nonconvex=False, offset=0.0, overall_maximum=None, overalL_minimum=None, lifetime=None, age=None, interest_rate=None, fixed_costs=None, custom_attributes=None), 'maximum': None, 'minimum': None, 'ep_costs': 116102.473, 'existing': None, 'nonconvex': None, 'offset': None, 'overall_maximum': None, 'overall_minimum': None, 'interest_rate': None, 'lifetime': None, 'variable_costs': None, 'max': None, 'min': None, 'fix': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,